In [1]:
from datetime import datetime
import os

# data_base_path = './duckEgg'
# if not os.path.isdir(data_base_path):
#     os.mkdir(data_base_path)

data_path = f'./data'
if not os.path.isdir(data_path):
    os.mkdir(data_path)

output_day_path = f'./duckEggPriceByDayCsv'
if not os.path.isdir(output_day_path):
    os.mkdir(output_day_path)
    
output_week_path = f'./duckEggPriceByWeekCsv'
if not os.path.isdir(output_week_path):
    os.mkdir(output_week_path)

n_years = 10

base_url = 'https://data.moa.gov.tw'
endpoint = '/api/v1/PoultryTransType_Goose_Duck_Duckegg'

# Include the API key in the headers
api_key = 'EXR1FU4WC5SNAUS0NXI0PM01JT2XRA'
headers = {'Authorization': f'Bearer {api_key}'}

# Get today's month and day
today = datetime.now()
formatted_month = today.strftime("%m") # Format the month and day with leading zeros if necessary
formatted_day = today.strftime("%d")
print(f"Today: {today}")

Today: 2023-11-28 00:06:49.309460


In [2]:
import requests
import json

years = [2023 - x for x in range(n_years)]

for year in years:
    print(f'===================================== {year} =====================================\n')
    
    if year == 2023:
        condition = f'/?Start_time={year}%2F01%2F01&End_time={year}%2F{formatted_month}%2F{formatted_day}'
    else:
        condition = f'/?Start_time={year}%2F01%2F01&End_time={year}%2F12%2F31'
    api_url = base_url + endpoint + condition

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        with open(f'{data_path}/{year}.txt', 'w') as file:
            json.dump(data, file)
#         print(data, "\n")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)


===================================== 2023 =====================================

===================================== 2022 =====================================

===================================== 2021 =====================================

===================================== 2020 =====================================

===================================== 2019 =====================================

===================================== 2018 =====================================

===================================== 2017 =====================================

===================================== 2016 =====================================

===================================== 2015 =====================================

===================================== 2014 =====================================



In [3]:
import json
import pandas as pd

def get_week_number(date_str):
    # Convert the input string to a datetime object
    date_object = datetime.strptime(date_str, "%Y/%m/%d")
    
    # Use strftime to format the date with the week number
    week_number = date_object.strftime("%U")
    
    week_num = '{:02d}'.format(int(week_number))
    return f'{date_str[:4]}-{week_num}'


# 到 2017（含）以前的格式跟近年不一樣，要調查並調整

target_column = 'Duckegg_TNN_TaijinPrice'

for year in years:
    print(f'===================================== {year} =====================================\n')

    with open(f"{data_path}/{year}.txt", 'r') as file:
        json_data = json.loads(file.read())

        df = pd.DataFrame(json_data['Data'])
        df = df[['TransDate', target_column]]

        # Change the format of the "TransDate" column to YYYY-MM-DD
        df_format_date = df.copy(deep=True)
        df_format_date['TransDate'] = pd.to_datetime(df_format_date['TransDate'])
        df_format_date['TransDate'] = df_format_date['TransDate'].dt.strftime('%Y-%m-%d')
        display(df_format_date)
        
        # drop duplicate-date rows
        df_format_date = df_format_date.drop_duplicates(subset=['TransDate'])
        
        df_format_date.to_csv(f'{output_day_path}/{year}DuckEggPriceByDay.csv', index=False)

        """ weekly avg price """
        numeric_price = []
        curr_price = 35
        for i, price in enumerate(df[target_column]):
            if price == '休市' or price == '議價':
                numeric_price.append(curr_price) # 往回找到最近一次有開市的價格
            else:
                numeric_price.append(float(price))
                curr_price = float(price)

        df[target_column] = numeric_price

        df['Year_Week'] = df['TransDate'].apply(get_week_number)
        df = df[['TransDate', 'Year_Week', target_column]]

        df = df.sort_values('Year_Week')
        display(df)

        df_week = df.groupby('Year_Week')[target_column].mean().reset_index()
        display(df_week)

        df_avg_price_by_week = pd.DataFrame({
            'Year_Week': df_week['Year_Week'],
            'Avg_Price': df_week[target_column]
        })

        df_avg_price_by_week['Avg_Price'] = df_avg_price_by_week['Avg_Price'].round(2)
        display(df_avg_price_by_week)
        
        df_avg_price_by_week.to_csv(f'{output_week_path}/{year}DuckEggPriceByWeek.csv', index=False)
        
    

===================================== 2023 =====================================



,TransDate,Duckegg_TNN_TaijinPrice
0,2023-11-26,48.0
1,2023-11-25,48.0
2,2023-11-24,48.0
3,2023-11-23,48.0
4,2023-11-22,48.0
...,...,...
326,2023-01-05,43.0
327,2023-01-04,43.0
328,2023-01-03,43.0
329,2023-01-02,43.0


,TransDate,Year_Week,Duckegg_TNN_TaijinPrice
330,2023/01/01,2023-01,43.0
324,2023/01/07,2023-01,43.0
325,2023/01/06,2023-01,43.0
329,2023/01/02,2023-01,43.0
327,2023/01/04,2023-01,43.0
...,...,...,...
4,2023/11/22,2023-47,48.0
3,2023/11/23,2023-47,48.0
2,2023/11/24,2023-47,48.0
1,2023/11/25,2023-47,48.0


,Year_Week,Duckegg_TNN_TaijinPrice
0,2023-01,43.000000
1,2023-02,43.000000
2,2023-03,43.000000
3,2023-04,43.000000
4,2023-05,43.000000
5,2023-06,43.000000
6,2023-07,43.000000
7,2023-08,44.714286
8,2023-09,45.000000
9,2023-10,46.285714


,Year_Week,Avg_Price
0,2023-01,43.00
1,2023-02,43.00
2,2023-03,43.00
3,2023-04,43.00
4,2023-05,43.00
5,2023-06,43.00
6,2023-07,43.00
7,2023-08,44.71
8,2023-09,45.00
9,2023-10,46.29


...

,TransDate,Year_Week,Duckegg_TNN_TaijinPrice
364,2018/01/01,2018-00,34.0
359,2018/01/06,2018-00,34.0
360,2018/01/05,2018-00,34.0
363,2018/01/02,2018-00,34.0
362,2018/01/03,2018-00,34.0
...,...,...,...
6,2018/12/25,2018-51,34.0
7,2018/12/24,2018-51,34.0
5,2018/12/26,2018-51,34.0
1,2018/12/30,2018-52,34.0


,Year_Week,Duckegg_TNN_TaijinPrice
0,2018-00,34.000000
1,2018-01,34.000000
2,2018-02,34.000000
3,2018-03,34.000000
4,2018-04,34.000000
5,2018-05,30.571429
6,2018-06,30.000000
7,2018-07,30.857143
8,2018-08,32.000000
9,2018-09,32.000000


,Year_Week,Avg_Price
0,2018-00,34.00
1,2018-01,34.00
2,2018-02,34.00
3,2018-03,34.00
4,2018-04,34.00
5,2018-05,30.57
6,2018-06,30.00
7,2018-07,30.86
8,2018-08,32.00
9,2018-09,32.00


===================================== 2017 =====================================



KeyError: "None of [Index(['TransDate', 'Duckegg_TNN_TaijinPrice'], dtype='object')] are in the [columns]"